In [ ]:
from __future__ import annotations

EXP = "exp001"
TARGET_MODEL = "/kaggle/input/mistral-7b-v0-1/Mistral-7B-v0.1"

DEBUG = False

In [ ]:
# %% Directory settings

# ====================================================
# Directory settings
# ====================================================
from pathlib import Path

OUTPUT_DIR = Path(f"/kaggle/input/llm-detect-{EXP}/")
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

INPUT_DIR = Path("../input/")

In [ ]:
import pandas as pd
test_df = pd.read_csv(INPUT_DIR / "llm-detect-ai-generated-text/test_essays.csv", sep=',')
print(f'test_df.shape: {test_df.shape}')


In [ ]:
test_df.head(3)

In [ ]:
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U accelerate --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/


In [ ]:
from transformers import AutoTokenizer, LlamaForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(TARGET_MODEL, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# datasets
from datasets import Dataset

# from pandas
test_ds = Dataset.from_pandas(test_df)


In [ ]:
def preprocess_function(examples, max_length=512):
    return tokenizer(examples["text"], truncation=True, max_length=max_length, padding=True)

In [ ]:
test_tokenized_ds = test_ds.map(preprocess_function, batched=True)


In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")

In [ ]:
# load model / tokenizer with 4bit bnb

from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType # type: ignore
from transformers import BitsAndBytesConfig
import torch


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
from transformers import AutoTokenizer, LlamaForSequenceClassification

base_model = LlamaForSequenceClassification.from_pretrained(
    TARGET_MODEL,
    num_labels=2,
    quantization_config=bnb_config,
    device_map={"":0}
)
base_model.config.pretraining_tp = 1 # 1 is 7b
base_model.config.pad_token_id = tokenizer.pad_token_id


In [ ]:
model = PeftModel.from_pretrained(base_model, str(OUTPUT_DIR))


In [ ]:
from transformers import TrainingArguments, Trainer

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
pred_output = trainer.predict(test_tokenized_ds)
logits = pred_output.predictions

In [ ]:
from scipy.special import expit as sigmoid
import numpy as np
def sigmoid(x):
    return 1 / (1 + np.exp(-x))  
probs = sigmoid(logits[:, 1])
probs

In [ ]:
sub = pd.DataFrame()
sub['id'] = test_df['id']
sub['generated'] = probs
sub.to_csv('submission.csv', index=False)
sub.head()